In [238]:
from typing import Iterable, Union
from random import random, randint


class skipnode:
    __slots__ = ("value", "nxt", "_down")

    def __init__(self, value=None, height=1):
        self.value = value
        self.nxt = [None] * height

    def __repr__(self):
        return f"{self.__class__.__qualname__}(value={self.value})"

    def __str__(self):
        return repr(self)

    def __next__(self):
        return self.nxt[0]


class skiplist:
    """ randomization at work. optimizing for kn^(1/ k). dT(n, k)/dk = 0 => k = ln(n)
        O(lg n) expected updates and queries w.h.p

        maintains a dynamic set of elements.
            methods: insert, search, delete, successor, predecessor
    """

    __slots__ = ("header", "count", "nil", "height", "maximum")

    MAXHEIGHT = 32
    INF = (1 << 63) - 1

    def __init__(self, items: Iterable = ()):
        self.count = 0
        self.header = self.nil = skipnode(self.INF, self.MAXHEIGHT)
        self.height = 1
        if items:
            self.insert(items)
        self.maximum = None

    @staticmethod
    def gen_height():
        c = 1
        while random() < 0.5 and c < skiplist.MAXHEIGHT:
            c += 1
        return c

    @staticmethod
    def genheight():
        x = int(random() * 0xffffffffff) & ((1 << skiplist.MAXHEIGHT) - 1)
        return (x & -x).bit_length()

    @property
    def isempty(self) -> bool:
        return self.count == 0

    def access(self, needle) -> Union[skipnode, None]:
        s = self.header
        for level in reversed(range(self.height)):
            while s.nxt[level] is not None and s.nxt[level].value <= needle:
                s = s.nxt[level]
                if s.value == needle:
                    return s
        return s

    def insert(self, *values) -> None:
        for value in values:
            h, H = skiplist.genheight(), self.height
            self.height = h if h > H else H
            elt = skipnode(value, h)

            s = self.header
            for level in reversed(range(h, self.height)):
                while s.nxt[level] and s.nxt[level].value < value:
                    s = s.nxt[level]

            for level in reversed(range(h)):
                while s.nxt[level] and s.nxt[level].value < value:
                    s = s.nxt[level]

                elt.nxt[level] = s.nxt[level]
                s.nxt[level] = elt

            self.count += 1
            self.maximum = value if not self.maximum or \
                        value > self.maximum else self.maximum
    
    @property
    def minimum():
        return None if self.isempty else self.header[0].nxt.value

    def delete(self, value) -> bool:
        target = s = self.header

        for level in reversed(range(self.height)):
            while target.nxt[level] and target.nxt[level].value < value:
                target = target.nxt[level]

        target = target.nxt[0]
        if not target or target.value != value:
            return False

        for level in reversed(range(self.height)):
            while s.nxt[level] and s.nxt[level].value < value:
                s = s.nxt[level]

            if s.nxt[level] == target:
                s.nxt[level] = target.nxt[level]
        self.count -= 1
        self.maximum = self._calc_max() if value == self.maximum else self.maximum
        return True
    
    def _calc_max():
        s = self.header.nxt[0]
        while s.next[0]:
            s = s.next[0]
        return s

    def successor(self, value) -> Union[skipnode, None]:
        p = self.predecessor(value)
        while p.nxt[0] != self.nil and p.nxt[0].value <= value:
            p = p.nxt[0]
        return p.nxt[0]

    def predecessor(self, value) -> skipnode:
        """If duplicate values exist, duplicate is returned"""
        target = self.header
        for level in reversed(range(self.height)):
            while target.nxt[level] and target.nxt[level].value < value:
                target = target.nxt[level]
        return target

    def __contains__(self, item):
        return self.access(item).value ==  self.nil.value

    def __len__(self):
        return self.count

    def __iter__(self):
        s = self.header.nxt[0]
        while s:
            yield s.value
            s = s.nxt[0]

    def __repr__(self):
        return f"{self.__class__.__qualname__}({str(self)})"

    def __str__(self):
        r = "["
        s = self.header.nxt[0]
        while s is not None:
            r += str(s.value) + ", "
            s = s.nxt[0]
        return r + "]"

    def __len__(self):
        return self.count

    def __iter__(self):
        s = self.header.nxt[0]
        while s:
            yield s.value
            s = s.nxt[0]

    def __repr__(self):
        return f"{self.__class__.__qualname__}({str(self)})"

    def __str__(self):
        r = "["
        s = self.header.nxt[0]
        while s is not None:
            r += str(s.value) + ", "
            s = s.nxt[0]
        return r + "]"